In [1]:
import numpy as np
import import_ipynb
import numpy.linalg as LA
import LMM as lmm
import random
import time
import os
import pandas as pd
from scipy.io import mmread
import scipy.sparse as sp
import matplotlib.pyplot as plt
from IPython.display import Image
import scanpy as sc
import matplotlib.pyplot as plt
from time import time as unix

np.random.seed(10)

importing Jupyter notebook from LMM.ipynb
LMM was imported successfully.


In [ ]:
##### Simulation of data
num_random_effects = 20
num_samples = 3000
num_fixed_effects = 0
num_features = 1000
sig_1 = 0.2
sig_2 = 0.8

# Generate data
X = np.ones((num_samples,1)) ### only intercepts
Z = np.random.normal(size=(num_samples,num_random_effects))
# Generate kernel
Z = (Z -  Z.mean(axis=0)) / Z.std(axis=0, ddof=1)
print(Z.shape)

K = Z @ Z.T / Z.shape[1]
I = np.eye(K.shape[0])
Kernel = [K]

# Generate phenotype assuming the generative process being y ~ MVN(Xb, V)
b = np.random.normal(size=(X.shape[1],1))
sigma = [sig_1, sig_2]
V = sigma[0] * K + sigma[1] * I
L = LA.cholesky(V)

# Sample from spherical normal dist
R = np.random.normal(size=(L.shape[1],num_features))
# r = MVN(0, I)

# reshape it to have the V covariance structure
Y = X @ b + L @ R
# L @ r ~ MVN(0, V)

In [4]:
def get_standard(Z):
    return (Z -  Z.mean(axis=0)) / Z.std(axis=0, ddof=1)

def sum_sigma_K(sigma, Kernel):
    sum_val = 0 
    for i in range(len(Kernel)):
        sum_val += sigma[i] * Kernel[i]
    return sum_val
        
##### Simulation of data

num_simulation = 5
num_random_effects = 4
num_samples = 200
num_fixed_effects = 0
num_features = 10

sig_1 = 0.15
sig_2 = 0.2
sig_3 = 0.25
sig_4 = 0.3
sig_res = 0.1



In [ ]:
# Generate data
X = np.ones((num_samples,1)) ### only intercepts

# Generate kernel
Z_1 = get_standard(np.random.normal(size=(num_samples,num_random_effects)))
Z_2 = get_standard(np.random.normal(size=(num_samples,num_random_effects)))
Z_3 = get_standard(np.random.normal(size=(num_samples,num_random_effects)))
Z_4 = get_standard(np.random.normal(size=(num_samples,num_random_effects)))

K_1 = Z_1 @ Z_1.T / Z_1.shape[1]
K_2 = Z_2 @ Z_2.T / Z_2.shape[1]
K_3 = Z_3 @ Z_3.T / Z_3.shape[1]
K_4 = Z_4 @ Z_4.T / Z_4.shape[1]

I = np.eye(K_1.shape[0])

Kernel = [K_1, K_2, K_3, K_4]
sigma = [sig_1, sig_2, sig_3, sig_4, sig_res]


# Generate phenotype assuming the generative process being y ~ MVN(Xb, V)
## b = np.random.normal(size=(X.shape[1],1)) ## equal b for all genes
b = np.random.normal(size=(X.shape[1],num_features)) ### different b for each gene

V = sum_sigma_K(sigma, Kernel) + sigma[len(sigma)-1] * I
L = LA.cholesky(V)

# Sample from spherical normal dist
R = np.random.normal(size=(L.shape[1],num_features))
# r = MVN(0, I)

# reshape it to have the V covariance structure
Y = X @ b + L @ R
# L @ r ~ MVN(0, V)

In [5]:
# Y shape now is 1000 by 1000 meaning that we have 10000 simulated reponse with covariance of V
print('X shape is: ', X.shape)
print('b shape is: ', b.shape)
print('Z shape is: ', Z_1.shape)
print('K shape is: ', K_1.shape)
print('V shape is: ', V.shape)
print('L shape is: ', L.shape)
print('R shape is: ', R.shape)
print('Y shape is: ', Y.shape)
print(type(Y))

NameError: name 'X' is not defined

In [5]:
simulation_results = []
simulation_runtime = []
total_start = unix()

for sim in range(num_simulation):
    
    # Generate data
    X = np.ones((num_samples,1)) ### only intercepts

    # Generate kernel
    Z_1 = get_standard(np.random.normal(size=(num_samples,num_random_effects)))
    Z_2 = get_standard(np.random.normal(size=(num_samples,num_random_effects)))
    Z_3 = get_standard(np.random.normal(size=(num_samples,num_random_effects)))
    Z_4 = get_standard(np.random.normal(size=(num_samples,num_random_effects)))

    K_1 = Z_1 @ Z_1.T / Z_1.shape[1]
    K_2 = Z_2 @ Z_2.T / Z_2.shape[1]
    K_3 = Z_3 @ Z_3.T / Z_3.shape[1]
    K_4 = Z_4 @ Z_4.T / Z_4.shape[1]

    I = np.eye(K_1.shape[0])

    Kernel = [K_1, K_2, K_3, K_4]
    sigma = [sig_1, sig_2, sig_3, sig_4, sig_res]


    # Generate phenotype assuming the generative process being y ~ MVN(Xb, V)
    ## b = np.random.normal(size=(X.shape[1],1)) ## equal b for all genes
    b = np.random.normal(size=(X.shape[1],num_features)) ### different b for each gene

    V = sum_sigma_K(sigma, Kernel) + sigma[len(sigma)-1] * I
    L = LA.cholesky(V)

    # Sample from spherical normal dist
    R = np.random.normal(size=(L.shape[1],num_features))
    # r = MVN(0, I)

    # reshape it to have the V covariance structure
    Y = X @ b + L @ R
    # L @ r ~ MVN(0, V)
    
    a_model_result = []
    results = []
    time_log = []

    start_time = unix()
    for i in range(Y.shape[1]): #
        a_model_result = ([-1], [-1], [-1], [-1])
        if Y.sum(axis=0)[i] != 0:
            a_model_result = lmm.reml_ei(Kernel, X, Y[:,i][:,np.newaxis]) 

        results.append(a_model_result)  

    execution_time = unix() - start_time
    
    simulation_results.append(results)
    simulation_runtime.append(execution_time)

total_runtime = unix() - total_start

Information: [[1.09905183e+02 7.79644215e-04 6.48332433e-04 5.42020655e-04
  2.29934190e+00]
 [7.79644215e-04 1.09692679e+02 2.51317686e-03 1.07277460e-03
  2.40075764e+00]
 [6.48332433e-04 2.51317686e-03 1.09660753e+02 1.28181983e-03
  2.41610036e+00]
 [5.42020655e-04 1.07277460e-03 1.28181983e-03 1.09793037e+02
  2.35315390e+00]
 [2.29934190e+00 2.40075764e+00 2.41610036e+00 2.35315390e+00
  5.24116320e+03]]
Score: [  -1.85103057  -12.48655166    8.57697529    7.48369304 -162.27043545]
	 EI REML vanilla round 0:   0.1159  0.0190  0.2110  0.2010  0.1012
Information: [[1.43507400e+02 2.42337158e-02 1.97129473e-04 1.83560596e-04
  2.99640019e+00]
 [2.42337158e-02 4.45322993e+03 2.37542204e-02 1.12878474e-02
  9.74761032e+01]
 [1.97129473e-04 2.37542204e-02 4.39777140e+01 1.31162061e-04
  9.62442068e-01]
 [1.83560596e-04 1.12878474e-02 1.31162061e-04 4.84732513e+01
  1.03621727e+00]
 [2.99640019e+00 9.74761032e+01 9.62442068e-01 1.03621727e+00
  8.94326628e+03]]
Score: [ 0.07108827  0.22

	 Estimates converged, exitting - diff: 0.000005
Information: [[3.39479065e+01 2.40819296e-04 2.00259243e-04 1.67421280e-04
  7.10228959e-01]
 [2.40819296e-04 3.38822675e+01 7.76279066e-04 3.31362458e-04
  7.41554614e-01]
 [2.00259243e-04 7.76279066e-04 3.38724059e+01 3.95933099e-04
  7.46293728e-01]
 [1.67421280e-04 3.31362458e-04 3.95933099e-04 3.39132662e+01
  7.26850602e-01]
 [7.10228959e-01 7.41554614e-01 7.46293728e-01 7.26850602e-01
  1.61890925e+03]]
Score: [  -3.08513028   -5.31423157    3.38151046   13.06588584 -219.47408024]
	 EI REML vanilla round 0:   0.1497  0.0839  0.3406  0.6259  0.1021
Information: [[8.67475225e+01 9.04928554e-04 4.74885562e-05 1.18266344e-05
  1.81609741e+00]
 [9.04928554e-04 2.69763568e+02 5.74407148e-04 7.29055886e-05
  5.91792514e+00]
 [4.74885562e-05 5.74407148e-04 1.70177901e+01 5.47834110e-06
  3.75356939e-01]
 [1.18266344e-05 7.29055886e-05 5.47834110e-06 5.06930505e+00
  1.08758514e-01]
 [1.81609741e+00 5.91792514e+00 3.75356939e-01 1.08758514

	 Estimates converged, exitting - diff: 0.000011
Information: [[2.71022330e+01 3.34163025e-04 2.85599032e-04 2.34232803e-04
  5.84735084e-01]
 [3.34163025e-04 2.70598675e+01 4.13393490e-04 3.84709144e-04
  6.05085236e-01]
 [2.85599032e-04 4.13393490e-04 2.70796580e+01 3.16885664e-04
  5.95563250e-01]
 [2.34232803e-04 3.84709144e-04 3.16885664e-04 2.70932030e+01
  5.89010294e-01]
 [5.84735084e-01 6.05085236e-01 5.95563250e-01 5.89010294e-01
  1.29414699e+03]]
Score: [  12.04164012   -4.90656228   -4.94643775    2.18080578 -204.2265601 ]
	 EI REML vanilla round 0:   0.7136  0.0881  0.0867  0.3498  0.1080
Information: [[3.90174567e+00 7.16943957e-05 6.31908956e-05 3.31750556e-06
  8.42009762e-02]
 [7.16943957e-05 2.44019946e+02 5.73023593e-03 3.41308374e-04
  5.46373443e+00]
 [6.31908956e-05 5.73023593e-03 2.51959025e+02 2.90215882e-04
  5.54753316e+00]
 [3.31750556e-06 3.41308374e-04 2.90215882e-04 1.61245408e+01
  3.50742832e-01]
 [8.42009762e-02 5.46373443e+00 5.54753316e+00 3.50742832

	 Estimates converged, exitting - diff: 0.000008
Information: [[5.75565455e+01 7.09656261e-04 6.06521745e-04 4.97436170e-04
  1.24179183e+00]
 [7.09656261e-04 5.74665746e+01 8.77916636e-04 8.17000184e-04
  1.28500910e+00]
 [6.06521745e-04 8.77916636e-04 5.75086033e+01 6.72964626e-04
  1.26478742e+00]
 [4.97436170e-04 8.17000184e-04 6.72964626e-04 5.75373687e+01
  1.25087102e+00]
 [1.24179183e+00 1.28500910e+00 1.26478742e+00 1.25087102e+00
  2.74835767e+03]]
Score: [  -0.91965311    1.37472626   11.74539778   -7.83158905 -179.93364854]
	 EI REML vanilla round 0:   0.1679  0.2079  0.3881  0.0478  0.1170
Information: [[6.88571424e+01 2.72796564e-04 6.77673946e-05 3.35383565e-03
  1.48583688e+00]
 [2.72796564e-04 4.51477210e+01 6.44430958e-05 3.63306266e-03
  1.00944266e+00]
 [6.77673946e-05 6.44430958e-05 1.31006328e+01 8.64924024e-04
  2.88072059e-01]
 [3.35383565e-03 3.63306266e-03 8.64924024e-04 7.89775984e+02
  1.71822002e+01]
 [1.48583688e+00 1.00944266e+00 2.88072059e-01 1.71822002

	 Estimates converged, exitting - diff: 0.000010
Information: [[5.84931860e+01 4.90030454e-04 8.08468609e-04 4.88290774e-04
  1.26618521e+00]
 [4.90030454e-04 5.84911662e+01 4.66426603e-04 8.65709002e-04
  1.26712767e+00]
 [8.08468609e-04 4.66426603e-04 5.85178431e+01 2.42522105e-04
  1.25429660e+00]
 [4.88290774e-04 8.65709002e-04 2.42522105e-04 5.85011721e+01
  1.26244119e+00]
 [1.26618521e+00 1.26712767e+00 1.25429660e+00 1.26244119e+00
  2.79347511e+03]]
Score: [  -5.83866322   10.45577095    5.26332067   -5.88035501 -170.40493051]
	 EI REML vanilla round 0:   0.0825  0.3611  0.2722  0.0818  0.1200
Information: [[2.76233567e+02 2.62987180e-04 7.57658809e-04 4.88479032e-03
  5.98326787e+00]
 [2.62987180e-04 1.51220634e+01 2.40955986e-05 4.72427264e-04
  3.27552082e-01]
 [7.57658809e-04 2.40955986e-05 2.64819336e+01 2.31735219e-04
  5.67768010e-01]
 [4.88479032e-03 4.72427264e-04 2.31735219e-04 2.80914966e+02
  6.06501856e+00]
 [5.98326787e+00 3.27552082e-01 5.67768010e-01 6.06501856

...

	 Estimates converged, exitting - diff: 0.000002
Information: [[7.73626164e+01 5.44258450e-04 8.78378796e-04 5.43811959e-04
  1.67120025e+00]
 [5.44258450e-04 7.74231875e+01 8.66456427e-04 5.25912240e-04
  1.64198942e+00]
 [8.78378796e-04 8.66456427e-04 7.73381442e+01 9.19226716e-04
  1.68272926e+00]
 [5.43811959e-04 5.25912240e-04 9.19226716e-04 7.73802190e+01
  1.66274763e+00]
 [1.67120025e+00 1.64198942e+00 1.68272926e+00 1.66274763e+00
  3.69428543e+03]]
Score: [  -8.30740658    3.90823009   -4.58102638    9.1229707  -222.487717  ]
	 EI REML vanilla round 0:   0.0513  0.2091  0.0995  0.2766  0.0972
Information: [[7.01351067e+02 1.08752324e-03 7.56893675e-03 6.26229284e-04
  1.51490142e+01]
 [1.08752324e-03 4.48370133e+01 4.79309738e-04 3.86312440e-05
  9.50841781e-01]
 [7.56893675e-03 4.79309738e-04 1.93843714e+02 2.92643957e-04
  4.21928134e+00]
 [6.26229284e-04 3.86312440e-05 2.92643957e-04 2.57550521e+01
  5.53540272e-01]
 [1.51490142e+01 9.50841781e-01 4.21928134e+00 5.53540272

	 Estimates converged, exitting - diff: 0.000038
Information: [[1.16460408e+02 8.19317706e-04 1.32229697e-03 8.18645566e-04
  2.51579733e+00]
 [8.19317706e-04 1.16551591e+02 1.30434923e-03 7.91699624e-04
  2.47182383e+00]
 [1.32229697e-03 1.30434923e-03 1.16423568e+02 1.38378876e-03
  2.53315292e+00]
 [8.18645566e-04 7.91699624e-04 1.38378876e-03 1.16486907e+02
  2.50307289e+00]
 [2.51579733e+00 2.47182383e+00 2.53315292e+00 2.50307289e+00
  5.56131644e+03]]
Score: [  -9.90002619   11.69576632   12.46489023   -7.88126411 -170.21661929]
	 EI REML vanilla round 0:   0.0439  0.2293  0.2360  0.0613  0.0976
Information: [[9.43732601e+02 1.22832938e-03 1.86770453e-03 1.64337684e-02
  2.03740479e+01]
 [1.22832938e-03 3.73697047e+01 7.34271318e-05 6.26786455e-04
  7.92185578e-01]
 [1.86770453e-03 7.34271318e-05 3.52694321e+01 1.04027173e-03
  7.66649488e-01]
 [1.64337684e-02 6.26786455e-04 1.04027173e-03 4.98019176e+02
  1.07003150e+01]
 [2.03740479e+01 7.92185578e-01 7.66649488e-01 1.07003150

	 Estimates converged, exitting - diff: 0.000027
Information: [[8.80397811e+01 6.07393434e-04 7.02341821e-04 7.52587496e-04
  1.88211139e+00]
 [6.07393434e-04 8.80387666e+01 8.83073257e-04 7.25560990e-04
  1.88250670e+00]
 [7.02341821e-04 8.83073257e-04 8.80385286e+01 5.40958540e-04
  1.88268238e+00]
 [7.52587496e-04 7.25560990e-04 5.40958540e-04 8.80032525e+01
  1.89966330e+00]
 [1.88211139e+00 1.88250670e+00 1.88268238e+00 1.89966330e+00
  4.20226006e+03]]
Score: [   1.11676855    8.13106734    1.84049882   -6.02443402 -175.50328552]
	 EI REML vanilla round 0:   0.1611  0.2408  0.1694  0.0800  0.1058
Information: [[7.49022702e+01 1.02147803e-04 2.36796740e-04 1.10407904e-03
  1.60213268e+00]
 [1.02147803e-04 3.38478562e+01 1.34528144e-04 4.81765719e-04
  7.24137138e-01]
 [2.36796740e-04 1.34528144e-04 6.78978809e+01 7.20241396e-04
  1.45307266e+00]
 [1.10407904e-03 4.81765719e-04 7.20241396e-04 2.95319955e+02
  6.37819464e+00]
 [1.60213268e+00 7.24137138e-01 1.45307266e+00 6.37819464

	 Estimates converged, exitting - diff: 0.000006
Information: [[3.67072088e+01 2.53245945e-04 2.92833620e-04 3.13782996e-04
  7.84725435e-01]
 [2.53245945e-04 3.67067858e+01 3.68187585e-04 3.02514595e-04
  7.84890258e-01]
 [2.92833620e-04 3.68187585e-04 3.67066865e+01 2.25546654e-04
  7.84963506e-01]
 [3.13782996e-04 3.02514595e-04 2.25546654e-04 3.66919786e+01
  7.92043509e-01]
 [7.84725435e-01 7.84890258e-01 7.84963506e-01 7.92043509e-01
  1.75208565e+03]]
Score: [   6.50890525   -8.3725127    -5.84820275   10.20909754 -240.79685021]
	 EI REML vanilla round 0:   0.4088  0.0034  0.0721  0.5097  0.0911
Information: [[1.18555744e+01 2.51504452e-02 1.47755491e-04 3.34022342e-06
  2.51427754e-01]
 [2.51504452e-02 7.07415094e+04 1.12154760e+00 1.84529053e-02
  1.50536555e+03]
 [1.47755491e-04 1.12154760e+00 3.64434422e+02 7.16862814e-05
  7.70123248e+00]
 [3.34022342e-06 1.84529053e-02 7.16862814e-05 7.63852955e+00
  1.63473533e-01]
 [2.51427754e-01 1.50536555e+03 7.70123248e+00 1.63473533

	 Estimates converged, exitting - diff: 0.000003


In [6]:
gene_sim_list = [np.empty((num_simulation,num_random_effects+2), dtype=object) for gene in range(num_features)]
print(gene_sim_list[0].shape)

sim_count = 0 
for sim in simulation_results:
    for a_gene_index in range(len(sim)):
        gene_sim_list[a_gene_index][sim_count,0] = sim[a_gene_index][0][0] ## sig 1
        gene_sim_list[a_gene_index][sim_count,1] = sim[a_gene_index][0][1] ## sig 2
        gene_sim_list[a_gene_index][sim_count,2] = sim[a_gene_index][0][2] ## sig 3
        gene_sim_list[a_gene_index][sim_count,3] = sim[a_gene_index][0][3] ## sig 4
        gene_sim_list[a_gene_index][sim_count,4] = sim[a_gene_index][0][4] ## sig res
        gene_sim_list[a_gene_index][sim_count,5] = sim[a_gene_index][len(sim[0])-1] # convergence
    
    sim_count += 1



(5, 6)


In [16]:

print('total_runtime is: ', str(total_runtime), ' seconds')
print('total_runtime is: ', str(total_runtime/60), ' mins')


total_runtime is:  1.1561639308929443  seconds


In [14]:
# print(gene_sim_list[0]) ### View the results for the first feature - list contains one matrix per gene
runtime_df = pd.DataFrame(simulation_runtime, columns=['run_time']) 
for i in range(num_features):
    a_gene_df = pd.DataFrame(gene_sim_list[i], columns=['sig1', 'sig2', 'sig3','sig4', 'sig_res', 'convergence']) 
    print(df.head)
    a_gene_df.to_csv('gene_' + str(i)'_simulation_' + str(num_samples) + '_numFeat' + str(num_features) + '.csv', 
              encoding='utf-8', index=False)
    

<bound method NDFrame.head of         sig1       sig2       sig3       sig4    sig_res convergence
0   0.116433  0.0190209   0.212847   0.199978   0.101137           0
1   0.711032  0.0888635  0.0872359   0.349156   0.108035           0
2  0.0819506    0.36058   0.273442  0.0815924    0.11996           0
3  0.0512139   0.208722  0.0990599   0.275882  0.0971823           0
4    0.16091   0.240621   0.169496  0.0798804    0.10578           0>
<bound method NDFrame.head of         sig1       sig2      sig3      sig4    sig_res convergence
0   0.121508   0.200392  0.134267  0.356221  0.0966322           0
1   0.224659   0.198505  0.101937  0.416232  0.0869959           0
2  0.0315973  0.0892519  0.399563  0.260961   0.103985           0
3   0.408895   0.373174  0.165252  0.356961   0.105087           0
4   0.135467  0.0647698  0.146971  0.172386  0.0874589           0>
<bound method NDFrame.head of         sig1       sig2       sig3       sig4    sig_res convergence
0  0.0224017   0.358316

In [33]:
execution_time/60

2.878591362635295

In [17]:
#### making a dict with gene names as keys and sigmas as the values 
sig_1_list = [a_model[0][0] for a_model in results]
sig_2_list = [a_model[0][1] for a_model in results]
convergence = [a_model[3] for a_model in results]
features_name = ['gene_' + str(i+1) for i in range(len(results))]

sig_df = pd.DataFrame(list(zip(features_name, sig_1_list, sig_2_list, convergence)),
               columns =['genes', 'sig_1', 'sig_2', 'convergence'])
sig_df = sig_df.sort_values(by='sig_1',axis=0,ascending=False)

sig_df.to_csv('sharedsig_sumSamp' + str(num_samples) + '_numFeat' + str(num_features) + '.csv', 
              encoding='utf-8', index=False)

<bound method NDFrame.head of       genes     sig_1     sig_2  convergence
34  gene_35  0.386142  0.799617            0
33  gene_34  0.374184  0.796247            0
37  gene_38  0.348778  0.813253            0
32  gene_33  0.324337  0.742365            0
41  gene_42  0.293393  0.843425            0
46  gene_47  0.272755  0.779395            0
11  gene_12  0.252373  0.776237            0
5    gene_6  0.248819  0.798171            0
20  gene_21  0.247500  0.786926            0
15  gene_16  0.247332  0.800693            0
14  gene_15  0.243609  0.796649            0
45  gene_46  0.241153  0.815475            0
35  gene_36  0.231405  0.799866            0
31  gene_32  0.228714  0.835075            0
23  gene_24  0.224405  0.764428            0
26  gene_27  0.213028  0.800455            0
30  gene_31  0.212938  0.797388            0
17  gene_18  0.211630  0.799261            0
8    gene_9  0.207255  0.801446            0
27  gene_28  0.207242  0.771595            0
24  gene_25  0.202987  0.

In [30]:

X = np.ones((num_samples, 1))  ### only intercepts

# Generate kernel
Z_1 = get_standard(np.random.normal(size=(num_samples, num_random_effects)))
Z_2 = get_standard(np.random.normal(size=(num_samples, num_random_effects)))

K_1 = Z_1 @ Z_1.T / Z_1.shape[1]
K_2 = Z_2 @ Z_2.T / Z_2.shape[1]

I = np.eye(K_1.shape[0])

Kernel = [K_1, K_2]
sigma = [sig_1, sig_2, sig_res]

# Generate phenotype assuming the generative process being y ~ MVN(Xb, V)
## b = np.random.normal(size=(X.shape[1],1)) ## equal b for all genes
b = np.random.normal(size=(X.shape[1], num_features))  ### different b for each gene

V = sum_sigma_K(sigma, Kernel) + sigma[len(sigma) - 1] * I
L = LA.cholesky(V)

# Sample from spherical normal dist
R = np.random.normal(size=(L.shape[1], num_features))
# r = MVN(0, I)

# reshape it to have the V covariance structure
Y = X @ b + L @ R
# L @ r ~ MVN(0, V)

a_model_result = []
results = []
time_log = []

start_time = unix()
for i in range(Y.shape[1]):  #
    a_model_result = ([-1], [-1], [-1], [-1])
    if Y.sum(axis=0)[i] != 0:
        a_model_result = lmm.reml_ei(Kernel, X, Y[:, i][:, np.newaxis],verbose=False)

    results.append(a_model_result)
    
print('----------------------------')
print(len(a_model_result))
print(a_model_result[0])
#print(a_model_result[1])
#print(a_model_result[2])
print(a_model_result[3])
print('----------------------------')
print(a_model_result)
a_model_result = np.array([0.81764203 ,1.24923772, 0.383054])



Information: [[7.71520260e+00 1.32797373e-04 1.62044517e-01]
 [1.32797373e-04 7.71518165e+00 1.62054234e-01]
 [1.62044517e-01 1.62054234e-01 3.82048110e+02]]
Score: [-2.50061709  2.50364376 -7.2736503 ]
Information: [[7.75917039e+02 3.28496441e-03 1.63059692e+01]
 [3.28496441e-03 2.13438355e+00 4.42678747e-02]
 [1.63059692e+01 4.42678747e-02 4.27242412e+02]]
Score: [ 0.13705257  0.00507632 -0.02079526]
Information: [[7.68337413e+02 3.22944513e-03 1.61467546e+01]
 [3.22944513e-03 2.11962799e+00 4.39646910e-02]
 [1.61467546e+01 4.39646910e-02 4.27381664e+02]]
Score: [-1.92403195e-03 -3.55286642e-05  2.40762903e-04]
Information: [[1.03547457e+01 1.78230321e-04 2.17483565e-01]
 [1.78230321e-04 1.03547176e+01 2.17496608e-01]
 [2.17483565e-01 2.17496608e-01 5.12755301e+02]]
Score: [  3.48834831  -2.367285   -18.09136032]
Information: [[2.38414487e+00 4.57971364e-04 4.98884730e-02]
 [4.57971364e-04 1.48210925e+02 3.11573660e+00]
 [4.98884730e-02 3.11573660e+00 6.56319892e+02]]
Score: [-0.0079

In [21]:
a_model_result = np.array([0.81764203 ,1.24923772, 0.383054])
print(a_model_result)
tesst = np.array([.0, .0, .0, .0])
print(tesst)
tesst[0:3] = a_model_result
tesst[3] = 98
print(tesst)

[0.81764203 1.24923772 0.383054  ]
[0. 0. 0. 0.]
[ 0.81764203  1.24923772  0.383054   98.        ]


In [6]:
import numpy as np
import import_ipynb
import numpy.linalg as LA
import LMM as lmm
import random
import time
import os
import pandas as pd
from scipy.io import mmread
import scipy.sparse as sp
import matplotlib.pyplot as plt
from IPython.display import Image
import scanpy as sc
import matplotlib.pyplot as plt
from time import time as unix

np.random.seed(10)


def get_standard(Z):
    return (Z - Z.mean(axis=0)) / Z.std(axis=0, ddof=1)


def sum_sigma_K(sigma, Kernel):
    sum_val = 0
    for i in range(len(Kernel)):
        sum_val += sigma[i] * Kernel[i]
    return sum_val


##### Simulation of data

num_simulation = 6
num_random_effects = 2
num_samples = 100
num_fixed_effects = 0
num_features = 10

sig_1 = 0.2
sig_2 = 0.5
sig_res = 0.3


simulation_result = np.empty((num_simulation, num_features, 4))
simulation_result.fill(-1.0)
runtimes = np.zeros(num_simulation)

total_start = unix()
for s_index in range(num_simulation):
    print('------------------------------------------------------------')
    print('simulation #', str(s_index), ' started...')

    # Generate data
    X = np.ones((num_samples, 1))  ### only intercepts

    # Generate kernel
    Z_1 = get_standard(np.random.normal(size=(num_samples, num_random_effects)))
    Z_2 = get_standard(np.random.normal(size=(num_samples, num_random_effects)))

    K_1 = Z_1 @ Z_1.T / Z_1.shape[1]
    K_2 = Z_2 @ Z_2.T / Z_2.shape[1]

    I = np.eye(K_1.shape[0])

    Kernel = [K_1, K_2]
    sigma = [sig_1, sig_2, sig_res]

    # Generate phenotype assuming the generative process being y ~ MVN(Xb, V)
    ## b = np.random.normal(size=(X.shape[1],1)) ## equal b for all genes
    b = np.random.normal(size=(X.shape[1], num_features))  ### different b for each gene

    V = sum_sigma_K(sigma, Kernel) + sigma[len(sigma) - 1] * I
    L = LA.cholesky(V)

    # Sample from spherical normal dist
    R = np.random.normal(size=(L.shape[1], num_features))
    # r = MVN(0, I)

    # reshape it to have the V covariance structure
    Y = X @ b + L @ R
    # L @ r ~ MVN(0, V)

    a_model_result = []
    results = []
    time_log = []

    start_time = unix()
    for gene_index in range(Y.shape[1]):  #
        a_model_result = ([-1], [-1], [-1], [-1])
        if Y.sum(axis=0)[gene_index] != 0:
            a_model_result = lmm.reml_ei(Kernel, X, Y[:, gene_index][:, np.newaxis],verbose=False)

        print(a_model_result[0])
        print('------------------------------------------------------------')
        simulation_result[s_index][gene_index][0:3] = a_model_result[0]
        simulation_result[s_index][gene_index][3] = a_model_result[3]

    execution_time = unix() - start_time
    print('------------------------------------------------------------')
    print('simulation #', str(s_index), ' execution time is ', str(execution_time), ' seconds')
    print('------------------------------------------------------------')
    
    runtimes[s_index] = execution_time

total_runtime = unix() - total_start


start...
------------------------------------------------------------
simulation # 0  started...
Information: [[2.37488465e+01 4.27781912e-05 4.92438098e-01]
 [4.27781912e-05 2.37613596e+01 4.86373812e-01]
 [4.92438098e-01 4.86373812e-01 1.17537224e+03]]
Score: [-2.23014662e+00 -2.89268840e-02  9.30342922e+01]
Information: [[8.07407188e+01 2.87067348e-04 1.67171096e+00]
 [2.87067348e-04 2.40461380e+01 4.91993052e-01]
 [1.67171096e+00 4.91993052e-01 6.04939707e+02]]
Score: [ 0.00975272  0.0150939  -0.0115905 ]
Information: [[8.05657196e+01 2.84651187e-04 1.66809336e+00]
 [2.84651187e-04 2.38987855e+01 4.88979237e-01]
 [1.66809336e+00 4.88979237e-01 6.05025944e+02]]
Score: [-1.32086404e-04 -3.28699374e-05  7.31722878e-05]
[0.10560292 0.19881942 0.28020531]
------------------------------------------------------------
Information: [[1.43985834e+01 2.59358009e-05 2.98558123e-01]
 [2.59358009e-05 1.44061699e+01 2.94881434e-01]
 [2.98558123e-01 2.94881434e-01 7.12611251e+02]]
Score: [ -1.4285

Score: [ 9.63405569e-07  1.31370908e-06 -5.37848408e-06]
[0.7520331 0.5920293 0.2791676]
------------------------------------------------------------
Information: [[5.26305169e+00 1.57048342e-05 1.07107138e-01]
 [1.57048342e-05 5.26303272e+00 1.07116168e-01]
 [1.07107138e-01 1.07116168e-01 2.60279594e+02]]
Score: [  0.95611587  -0.6083724  -30.19888328]
Information: [[2.62205450e+00 7.68592927e-06 5.33610388e-02]
 [7.68592927e-06 9.74727137e+00 1.98410308e-01]
 [5.33610388e-02 1.98410308e-01 4.90649967e+02]]
Score: [ 5.64844804e-06 -4.97956244e-03  5.03110564e-03]
Information: [[2.62203620e+00 7.71096315e-06 5.33606412e-02]
 [7.71096315e-06 9.77843893e+00 1.99044733e-01]
 [5.33606412e-02 1.99044733e-01 4.90616991e+02]]
Score: [ 4.47332892e-06 -3.21667713e-06 -5.54943080e-06]
[0.61123063 0.31345571 0.31115572]
------------------------------------------------------------
Information: [[2.34624649e+01 7.00114957e-05 4.77479153e-01]
 [7.00114957e-05 2.34623803e+01 4.77519408e-01]
 [4.77479

Information: [[1.33510349e+03 3.12897332e-03 2.79830538e+01]
 [3.12897332e-03 3.35241334e+00 6.91449931e-02]
 [2.79830538e+01 6.91449931e-02 8.85795012e+02]]
Score: [-0.0639178   0.00030342  0.00566647]
[0.0224571  0.54145321 0.23165503]
------------------------------------------------------------
Information: [[4.39170767e+01 5.60806877e-04 9.24099456e-01]
 [5.60806877e-04 4.39380401e+01 9.13962894e-01]
 [9.24099456e-01 9.13962894e-01 2.17488206e+03]]
Score: [ -2.99220722  -5.0545562  441.55441768]
Information: [[1.46150309e+02 1.83951614e-01 3.03927731e+00]
 [1.83951614e-01 7.79846058e+02 1.61501620e+01]
 [3.03927731e+00 1.61501620e+01 3.86195461e+02]]
Score: [-0.27467073 -0.18844068  0.04052059]
Information: [[1.53026055e+02 1.95310663e-01 3.18185636e+00]
 [1.95310663e-01 7.90479633e+02 1.63684385e+01]
 [3.18185636e+00 1.63684385e+01 3.85917654e+02]]
Score: [-0.00682873 -0.00493463  0.00180747]
[0.07345407 0.02827347 0.35102655]
------------------------------------------------------

Information: [[5.82571161e+01 9.72616215e-05 1.22548144e+00]
 [9.72616215e-05 1.42425415e+00 2.96290524e-02]
 [1.22548144e+00 2.96290524e-02 4.95484432e+02]]
Score: [ 1.76252881e-04  2.81581685e-05 -1.46597029e-04]
[0.1245029  0.831497   0.30963009]
------------------------------------------------------------
Information: [[1.08948851e+01 1.27313552e-04 2.29170264e-01]
 [1.27313552e-04 1.08992069e+01 2.27071844e-01]
 [2.29170264e-01 2.27071844e-01 5.39531979e+02]]
Score: [-0.61101068 -1.19129331 61.27670886]
Information: [[1.64076082e+01 8.97609271e-04 3.44576385e-01]
 [8.97609271e-04 2.66805225e+01 5.55322688e-01]
 [3.44576385e-01 5.55322688e-01 2.82090336e+02]]
Score: [ 0.020392    0.02749551 -0.01822762]
Information: [[1.62436431e+01 8.78959603e-04 3.41138722e-01]
 [8.78959603e-04 2.63987252e+01 5.49463779e-01]
 [3.41138722e-01 5.49463779e-01 2.82183872e+02]]
Score: [-0.0002206  -0.0003084   0.00026938]
[0.23947591 0.18606915 0.41029532]
---------------------------------------------

In [9]:
print(simulation_result)
print(runtimes)


[[[ 0.10560292  0.19881942  0.28020531  0.        ]
  [ 0.1594889   0.39829158  0.23970709  0.        ]
  [ 0.10866398  1.77184368  0.26064571  0.        ]
  [ 0.01959552  0.34607079  0.3130975   0.        ]
  [ 0.06818705  0.09453698  0.31184638  0.        ]
  [ 0.47444219  1.23679285  0.25207501  0.        ]
  [ 0.334384    0.01659152  0.29427535  0.        ]
  [ 0.33419049  0.21135374  0.27217129  0.        ]
  [-0.00373352  0.32263327  0.28655312  0.        ]
  [ 0.07207274  0.12390141  0.21956582  0.        ]]

 [[ 0.3012373   0.12440175  0.31957865  0.        ]
  [ 0.01961551  0.22264903  0.27457692  0.        ]
  [ 0.46585707  0.16150378  0.28798895  0.        ]
  [ 0.12635609  0.41398752  0.25768694  0.        ]
  [ 0.40926323  0.8634994   0.29608938  0.        ]
  [ 0.23424451  0.61543829  0.2802627   0.        ]
  [ 0.7520331   0.5920293   0.2791676   0.        ]
  [ 0.61123063  0.31345571  0.31115572  0.        ]
  [ 0.22626846  0.0616752   0.32437874  0.        ]
  [ 0.3227

In [3]:
folder_name = 'sim_' + str(num_simulation)+ '_numFeat_'+ str(num_features)+'_numSamp_' str(num_samples)  
gene_sim_list = [np.empty((num_simulation, num_random_effects + 2), dtype=object) for gene in range(num_features)]
print(gene_sim_list[0].shape)

sim_count = 0
for sim in simulation_results:
    for a_gene_index in range(len(sim)):
        gene_sim_list[a_gene_index][sim_count, 0] = sim[a_gene_index][0][0]  ## sig 1
        gene_sim_list[a_gene_index][sim_count, 1] = sim[a_gene_index][0][1]  ## sig 2
        gene_sim_list[a_gene_index][sim_count, 2] = sim[a_gene_index][0][2]  ## sig res
        gene_sim_list[a_gene_index][sim_count, 3] = sim[a_gene_index][len(sim[0]) - 1]  # convergence

    sim_count += 1


print('total_runtime is: ', str(total_runtime), ' seconds')
print('total_runtime is: ', str(total_runtime/60), ' mins')

# print(gene_sim_list[0]) ### View the results for the first feature - list contains one matrix per gene
runtime_df = pd.DataFrame(simulation_runtime, columns=['run_time'])
runtime_df.to_csv('runtime_info_' + str(num_simulation) +'_simulation_' + str(num_samples) + '_numFeat' + str(num_features) + '.csv',encoding='utf-8', index=False)

for i in range(num_features):
    a_gene_df = pd.DataFrame(gene_sim_list[i], columns=['sig1', 'sig2', 'sig_res', 'convergence'])
    a_gene_df.to_csv(folder_name + '/' + 'gene_' + str(i) + 'sim_' + str(num_simulation) + 
                     '_numFeat_'+ str(num_features) + '_numSamp_' + str(num_samples) + '.csv', 
                     encoding = 'utf-8', index = False)




(20, 4)
total_runtime is:  48.86295199394226  seconds
total_runtime is:  0.814382533232371  mins


In [11]:
folder_name = 'sim_' + str(num_simulation)+ '_numFeat_'+ str(num_features)+'_numSamp_' + str(num_samples)  
print(folder_name)
i = 0
num_samples = 10
print(folder_name+'/' + 'gene_' + str(i) + 'sim_' + str(num_simulation)+ '_numFeat_'+ str(num_features)+
      '_numSamp_' + str(num_samples) + '.csv')


sim_20_numFeat_10_numSamp_10
sim_20_numFeat_10_numSamp_10/gene_0sim_20_numFeat_10_numSamp_10.csv


In [13]:
num_samples

10